In [1]:
%run "../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [2]:
schema = {
  "type": "object",
  "properties": {
    "ingredients": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": { "type": "string" },
          "unit": { 
            "type": "string",
            "enum": ["grams", "ml", "cups", "pieces", "teaspoons"]
          },
          "amount": { "type": "number" }
        },
        "required": ["name", "unit", "amount"]
      }
    },
    "instructions": {
      "type": "array",
      "description": "Steps to prepare the recipe (no numbering)",
      "items": { "type": "string" }
    },
    "time_to_cook": {
      "type": "number",
      "description": "Total time to prepare the recipe in minutes"
    }
  },
  "required": ["ingredients", "instructions", "time_to_cook"]
}

In [4]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Provide a recipe for bibimbap"}
  ],
  functions=[{"name": "set_recipe", "parameters": schema}],
  function_call={"name": "set_recipe"},
  temperature=0,
)

print(completion.choices[0].message.function_call.arguments)

{
  "ingredients": [
    {
      "name": "cooked rice",
      "unit": "cups",
      "amount": 2
    },
    {
      "name": "spinach",
      "unit": "grams",
      "amount": 200
    },
    {
      "name": "carrot",
      "unit": "pieces",
      "amount": 1
    },
    {
      "name": "zucchini",
      "unit": "pieces",
      "amount": 1
    },
    {
      "name": "shiitake mushrooms",
      "unit": "grams",
      "amount": 100
    },
    {
      "name": "ground beef",
      "unit": "grams",
      "amount": 150
    },
    {
      "name": "soy sauce",
      "unit": "ml",
      "amount": 30
    },
    {
      "name": "sesame oil",
      "unit": "ml",
      "amount": 30
    },
    {
      "name": "garlic",
      "unit": "teaspoons",
      "amount": 2
    },
    {
      "name": "eggs",
      "unit": "pieces",
      "amount": 2
    },
    {
      "name": "gochujang (Korean hot pepper paste)",
      "unit": "teaspoons",
      "amount": 2
    }
  ],
  "instructions": [
    "Cook the rice accordi

In [ ]:
%pip install scipy
%pip install tenacity
%pip install tiktoken==0.3.3
%pip install termcolor 
%pip install openai
%pip install requests
%pip install arxiv
%pip install pandas
%pip install PyPDF2
%pip install tqdm

In [ ]:
import arxiv
import ast
import concurrent
from csv import writer
from IPython.display import display, Markdown, Latex
import json
import os
import pandas as pd
from PyPDF2 import PdfReader
import requests
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
from tqdm import tqdm
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [ ]:
import requests

def get_collections():
    response = requests.get('https://api.dc.library.northwestern.edu/api/v2/collections?size=10000')

    if response.status_code != 200:
        print(f"Failed to get data from the API. Status code: {response.status_code}")
        return None
    
    print(len(response.json()['data']))

    for collection in response.json()['data']:
        title = collection['title']
        description = collection['description']
        print(f'Title: {title}\nDescription: {description}\n')

    # return collections['data']

In [ ]:
get_collections()

In [ ]:
def search_collection(collection_name):
    """
    Perform a search on DC API to get a collection based on the title.

    :param collection_name: The name of the collection.
    :return: The collection object or some representative data of the collection.
    """
    # The actual implementation will depend on the specifics of your DC API.
    # Replace this with appropriate API interaction logic.
    api_response = your_dc_api.search_collection(collection_name)

    return api_response


In [ ]:
dc_functions = [
    {
        "name": "search_collection",
        "description": "Search for a collection in the DC API based on the title.",
        "parameters": {
            "type": "object",
            "properties": {
                "collection_name": {
                    "type": "string",
                    "description": "The name of the collection to search for.",
                },
            },
            "required": ["collection_name"],
        },
    },
    {
        "name": "semantic_search",
        "description": "Search for a collection in the DC API based on the title.",
        "parameters": {
            "type": "object",
            "properties": {
                "collection_name": {
                    "type": "string",
                    "description": "The name of the collection to search for.",
                },
            },
            "required": ["collection_name"],
        },
    },
]
]


In [ ]:
def call_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    function_name = full_message["message"]["function_call"]["name"]
    parsed_output = json.loads(full_message["message"]["function_call"]["arguments"])

    if function_name == "search_collection":
        print("Searching for collection")
        content = search_collection(parsed_output["collection_name"])
    elif function_name == "semantic_search":
        print("Query the vector database")
        content = summarize_text(parsed_output["query"])
    else:
        raise Exception("Function does not exist and cannot be called")

    messages.append({
        "role": "function",
        "name": function_name,
        "content": content,
    })

    try:
        print("Processing results")
        response = chat_completion_request(messages)
        return response.json()
    except Exception as e:
        print(type(e))
        raise Exception("Function chat request failed")
